<a href="https://colab.research.google.com/github/Foluwa/Extending-AfriXNLI/blob/main/yoruba/AfriXNLI_English_Yoruba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd
from tqdm.auto import tqdm
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [2]:
!nvidia-smi

Mon May 19 19:22:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             47W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!apt-get install -y git-lfs
!git lfs install
!git clone https://huggingface.co/datasets/masakhane/afrixnli /content/masakhane/afrixnli

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Git LFS initialized.
Cloning into '/content/masakhane/afrixnli'...
remote: Enumerating objects: 153, done.
remote: Total 153 (delta 0), reused 0 (delta 0), pack-reused 153 (from 1)
Receiving objects: 100% (153/153), 1.02 MiB | 1.56 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [4]:
train_path = "./masakhane/afrixnli/data/eng/dev.tsv"
test_path  = "./masakhane/afrixnli/data/eng/test.tsv"

# Load data set
train_df = pd.read_csv(train_path, sep="\t", header=0)
test_df  = pd.read_csv(test_path,  sep="\t", header=0)

print(f"Train DataFrame shape: {train_df.shape}")
print(f"Test  DataFrame shape: {test_df.shape}")

Train DataFrame shape: (450, 3)
Test  DataFrame shape: (600, 3)


In [5]:
train_df.head()

,premise,hypothesis,label
0,"And he said, Mama, I'm home.",He called his mom as soon as the school bus dr...,1
1,"And he said, Mama, I'm home.",He didn't say a word.,2
2,"And he said, Mama, I'm home.",He told his mom he had gotten home.,0
3,I didn't know what I was going for or anything...,I have never been to Washington so when I was ...,1
4,I didn't know what I was going for or anything...,I knew exactly what I needed to do as I marche...,2


In [6]:
test_df.head()

,premise,hypothesis,label
0,"Well, I wasn't even thinking about that, but I...",I havent spoken to him again.,2
1,"Well, I wasn't even thinking about that, but I...",I was so upset that I just started talking to ...,0
2,"Well, I wasn't even thinking about that, but I...",We had a great talk.,1
3,"They told me that, uh, that I would be called ...",I was never told anything about meeting anyone.,2
4,"They told me that, uh, that I would be called ...",I was told a guy would be called in for me to ...,0


# [HelpMumHQ/AI-translator-eng-to-9ja](https://huggingface.co/HelpMumHQ/AI-translator-eng-to-9ja)

In [8]:
MODEL_NAME_M2M100 = "HelpMumHQ/AI-translator-eng-to-9ja"

# Load tokenizer & model
translator_tokenizer_m2m100 = M2M100Tokenizer.from_pretrained(MODEL_NAME_M2M100)
translator_model_m2m100 = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME_M2M100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
translator_model_m2m100.to(device)

tokenizer_config.json:   0%|          | 0.00/19.8k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/938 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [9]:
def translate_texts_m2m100(
    texts,
    src_lang="en",
    tgt_lang="yo",
    batch_size=16,
    max_length=128,
    num_beams=4
):
    """
    Translate `texts` (list of strings) from `src_lang` to `tgt_lang`
    using the globally-loaded M2M-100 model.
    Returns a list[str] of the same length.
    """
    tok = translator_tokenizer_m2m100
    mdl = translator_model_m2m100
    tok.src_lang = src_lang
    decoded_all = []

    for i in tqdm(range(0, len(texts), batch_size), desc=f"M2M100 {src_lang}→{tgt_lang}"):
        batch = texts[i : i + batch_size]
        enc = tok(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_length
        ).to(device)

        gen = mdl.generate(
            **enc,
            forced_bos_token_id=tok.get_lang_id(tgt_lang),
            max_length=max_length,
            num_beams=num_beams
        )
        decoded_all.extend(tok.batch_decode(gen, skip_special_tokens=True))

    return decoded_all


In [10]:
TEXT_COLS = ["premise", "hypothesis"]
for col in TEXT_COLS:
    for df in (train_df, test_df):
        new_col = f"{col}_yoruba"
        df_type = "train" if df is train_df else "test"
        print(f"Translating `{col}` → `{new_col}` in {df_type}…")
        df[new_col] = translate_texts_m2m100(
            df[col].astype(str).tolist(),
            src_lang="en",
            tgt_lang="yo",
            batch_size=16
        )

Translating `premise` → `premise_yoruba` in train…


M2M100 en→yo:   0%|          | 0/29 [00:00<?, ?it/s]

Translating `premise` → `premise_yoruba` in test…


M2M100 en→yo:   0%|          | 0/38 [00:00<?, ?it/s]

Translating `hypothesis` → `hypothesis_yoruba` in train…


M2M100 en→yo:   0%|          | 0/29 [00:00<?, ?it/s]

Translating `hypothesis` → `hypothesis_yoruba` in test…


M2M100 en→yo:   0%|          | 0/38 [00:00<?, ?it/s]

In [11]:
train_df.to_csv("train_translated_m2m100.tsv", sep="\t", index=False)
test_df.to_csv("test_translated_m2m100.tsv",  sep="\t", index=False)
print("Files written: train_translated_m2m100.tsv, test_translated_m2m100.tsv")

Files written: train_translated_m2m100.tsv, test_translated_m2m100.tsv


In [12]:
train_dfe = pd.read_csv('./train_translated_m2m100.tsv', sep="\t", header=0)

In [13]:
train_dfe.head()

,premise,hypothesis,label,premise_yoruba,hypothesis_yoruba
0,"And he said, Mama, I'm home.",He called his mom as soon as the school bus dr...,1,"O si wipe, Mama, Mo wa ni ile.",O pe iya rẹ lẹsẹkẹsẹ nigbati ọkọ ayọkẹlẹ ti fi...
1,"And he said, Mama, I'm home.",He didn't say a word.,2,"O si wipe, Mama, Mo wa ni ile.",O ko sọ ọrọ kankan.
2,"And he said, Mama, I'm home.",He told his mom he had gotten home.,0,"O si wipe, Mama, Mo wa ni ile.",Ó sọ fún ìyá rẹ̀ pé ó ti dé ilé.
3,I didn't know what I was going for or anything...,I have never been to Washington so when I was ...,1,"Emi ko mọ ohun ti Mo nilo lati ṣe, tabi ohunko...",Emi ko ti lọ si Washington nitorina nigbati mo...
4,I didn't know what I was going for or anything...,I knew exactly what I needed to do as I marche...,2,"Emi ko mọ ohun ti Mo nilo lati ṣe, tabi ohunko...",Mo mọ ohun ti Mo nilo lati ṣe nigbati mo lọ si...


In [14]:
test_dfe = pd.read_csv('./test_translated_m2m100.tsv', sep="\t", header=0)